In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch
import numpy as np
import json

In [3]:
path = r"C:\Users\burak.tanriver\Downloads\richtig.csv"
df = pd.read_csv(path, sep= ";",dtype=str, encoding="UTF-8", usecols=["label", "queryterm", "tokens_suggestions_cleaned_root", "tokens_suggestions_tensor"])

In [4]:
tensordict = {}
count = 0
for index, row in df.iterrows():
    #try:
    tensorlfind = str(row["tokens_suggestions_tensor"]).find("[")
    tensorrfind = str(row["tokens_suggestions_tensor"]).find("]")
    tensorlist = str(row["tokens_suggestions_tensor"])[tensorlfind+1:tensorrfind].replace("\n", "").replace(" ", "").split(",")
    if not row["label"] in tensordict.keys():
        tensordict[row["label"]] = {}
    if not row["queryterm"] in tensordict.keys():
        tensordict[row["label"]][row["queryterm"]]= {}
    #except AttributeError:
        #continue
    for index2, tensorname in enumerate(row["tokens_suggestions_cleaned_root"][1:-1].replace(r"'", "").split(","), start = 0):
        tensordict[row["label"]][row["queryterm"]][tensorname.strip()] = tensorlist[index2]
with open("tensordict.txt","w",encoding="UTF-8") as f:
    f.write(json.dumps(tensordict,ensure_ascii=False))
    f.close()


In [6]:
searchterms = ["judentum","christentum","islam"]
resdf = pd.DataFrame(columns=["engine","searchterm","count_neg_tensor","tensorminname","tensormin"])
with open("tensordict.txt","r",encoding="UTF-8") as f:
    tensorjson = json.load(f)
for firstlevel in tensorjson.keys():
    for key in tensorjson[firstlevel].keys():
        for x in searchterms:
            if x == key:
                count = 0
                for tensor, value in tensorjson[firstlevel][key].items():
                    if value.startswith("-"):
                        tensormin = float()
                        if float(value) < tensormin:
                            tensormin = float(value)
                            tensorminname = tensor
                        count +=1
                resdf = resdf.append(pd.DataFrame([[firstlevel,key,count,tensorminname,tensormin]], columns=resdf.columns))
                print(firstlevel,key,count,tensorminname,tensormin)
resdf.to_csv("tensordict.csv", index=False)

google_psy judentum 9 symbole -0.78496
google_psy christentum 7 englisch -0.89272
google_psy islam 7 islamischer -0.90637
google_firefox judentum 9 symbole -0.78496
google_firefox christentum 7 englisch -0.89272
google_firefox islam 7 islamischer -0.90637
ddg judentum 7 religion -0.0021757
ddg christentum 5 translate -0.6374
ddg islam 6 islamabad -0.8516
bing judentum 7 sachsen -0.53646
bing christentum 9 kurzfassung -0.70721
bing islam 13 staat -0.75809
